# Install GRASS GIS

In [ ]:
%%bash
DEBIAN_FRONTEND=noninteractive sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable && apt update && apt install grass xvfb
pip install PyVirtualDisplay
git clone https://github.com/ncsu-geoforall-lab/grass-gis-workshop-vanderbilt-2024
cd grass-gis-workshop-vanderbilt-2024
sh download_data.sh

In [ ]:
import os
os.chdir("grass-gis-workshop-vanderbilt-2024")

# Part 1: GRASS GIS intro

In this first part, we will demonstrate starting GRASS GIS, creating new project and basic data visualization.

## Notebook environment

By default all cells in a notebook are running Python:

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

Cells can also run Bash using IPython magic. Later, we will use this to show how GRASS can be used from command line (this assumes that the machine has Bash).

In [ ]:
%%bash
pwd

We can use ! to run individual lines in command line (we won't use it in this notebook, but this works on all machines even if they don't have Bash).

In [ ]:
!pwd

## Start GRASS GIS

First, we create new empty project (location) called *dix_park* that uses projection [UTM zone 17 N](https://epsg.io/6346) with EPSG:6346.

Flag `c` stands for _creating_ new project and `e` will _exit_ the command after creating the project. See [manual](https://grass.osgeo.org/grass-stable/manuals/grass.html) for more examples.

In [ ]:
%%bash
grass -c EPSG:6346 -e ~/grassdata/dix_park/ 

Start GRASS GIS session in the newly created project. Load Python libraries.

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("~/grassdata/dix_park/")

## Import data

<div class="alert alert-info">
If you run this workshop online, data is already prepared in the environment. Otherwise download the data <a href="https://zenodo.org/records/6967699/files/FOSS4G_2022_GRASS_GIS_workshop.zip?download=1">here</a>.
</div>

We will import prepared digitial surface model (DSM), bare ground (digital terrain model, DTM) and ortho maps. 
The data CRS matches the CRS of the *dix_park* location, so we don't need to reproject it.

In [ ]:
%%bash
r.import input=dsm.tif output=dsm resample=bilinear
r.import input=ground.tif output=ground
r.import input=ortho.tif output=ortho

Next, we will import pre-downloaded OSM data of roads restricted to our study area. We obtained the roads using Overpass Turbo with this [query](https://overpass-turbo.eu/s/1aGu) and exported to GeoJSON. 

The data comes in EPSG:4326, so it will be automatically reprojected to UTM during the import.

In [ ]:
%%bash
v.import input=roads.geojson output=roads

Let's look at the available data in our location:

In [ ]:
%%bash
g.list type=raster,vector -m -t

Schema of GRASS project dix_park's content:

<img src="https://github.com/ncsu-geoforall-lab/grass-gis-workshop-vanderbilt-2024/blob/main/img/data_structure1.png?raw=true" alt="GRASS project dix_park" width="200"/>

## Data visualization

We will call _d.rast_ and _d.vect_ modules using the [GRASS Jupyter API](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html). The `Map` class creates and displays GRASS maps as static PNG images.

First let's display bare ground elevation with the roads dataset:

In [ ]:
# Create Map instance
elevation_map = gj.Map()
# Add a raster and vector to the map
elevation_map.d_rast(map="ground")
elevation_map.d_vect(map="roads")
elevation_map.d_legend(raster="ground")
# Display map
elevation_map.show()

Next, display road _Umstead Drive_ in yellow on top of ortho. Method `d_vect` can be called with options of [d.vect](https://grass.osgeo.org/grass-stable/manuals/d.vect.html).

In [ ]:
ortho_map = gj.Map()
ortho_map.d_rast(map="ortho")
ortho_map.d_vect(map="roads", width="2", color="yellow", where="name = 'Umstead Drive'")
ortho_map.show()

Here is how we can visualize data interactively with folium:

In [ ]:
roads_map = gj.InteractiveMap()
roads_map.add_raster("dsm", opacity=0.5)
roads_map.add_vector("roads")
roads_map.add_layer_control(position="bottomright")
roads_map.show()

We can also visualize data in 3D. Here we drape the ortho over the DSM.


In [ ]:
map3d = gj.Map3D()
map3d.render(
    elevation_map="dsm",
    color_map="ortho",
    position=(0.5, 1),
    height=3000,
    perspective=12,
)
map3d.show()

In [ ]:
map3d = gj.Map3D()
map3d.render(
    elevation_map="dsm",
    resolution_fine=1,
    color_map="ortho",
    light_position=(1, 0, 0.5),
    position=(0.75, 0.35),
    height=1500,
    perspective=10,
)
map3d.show()

## GRASS GIS tools

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

There is a tool for finding other tools:

In [ ]:
%%bash
g.search.modules keyword=zonal

Here is how to get all options and flags of a GRASS tool:

In [ ]:
%%bash
r.univar --help

This will open the tool's manual page in your web browser. It will work only locally. The page is available [online](https://grass.osgeo.org/grass83/manuals/r.univar.html) for each version.

```
g.manual r.univar
```

GRASS modules can be executed either through the GUI, command line or Python interfaces. This is an example how to execute a tool in command line. Specifically, it will extract road _Umstead Drive_ into a new vector `umstead_drive_segments`.

In [ ]:
%%bash
v.extract input=roads where="name = 'Umstead Drive'" output=umstead_drive_segments

Now the same in Python:

In [ ]:
gs.run_command(
    "v.extract",
    input="roads",
    where="name = 'Umstead Drive'",
    output="umstead_drive_segments",
)

## Computational region

Computational region is an important raster concept in GRASS GIS, that allows you to fully control the **extent** and **resolution** of your raster computations.
All raster computations will be performed in the specified extent and with the given resolution to ensure consistency.
Among other things, computational region allows us to easily subset larger extent data for quicker testing of analysis, or to run an analysis of specific regions given by e.g. administrative units.

A few points to keep in mind:

 * computational region is defined by extent and resolution
 * applies to all raster operations and vector to raster operations
 * persists between GRASS sessions, can be different for different mapsets (subprojects)
 * advantages: keeps your results consistent, avoids clipping, facilitates experimentation (for computationally demanding tasks set region to smaller extent, check that your result is good and then set the computational region to the entire study area and rerun analysis)

Run `g.region -p` to see current region settings:
 

In [ ]:
%%bash
g.region -p

The most common way to set region is **based on a raster map** - both extent and resolution. Run again g.region (we include -p flag to always see the resulting region):

<img src="https://github.com/ncsu-geoforall-lab/grass-gis-workshop-vanderbilt-2024/blob/main/img/region_raster.png?raw=true" alt="Region set to raster" width="200"/>

In [ ]:
%%bash
g.region -p raster=dsm

Computational region can be set also **using a vector map**. In that case, only extent is set (as vector maps do not have any resolution - at least not in the way raster maps do). The resolution needs to be adjusted based on the new extent:

<img src="https://github.com/ncsu-geoforall-lab/grass-gis-workshop-vanderbilt-2024/blob/main/img/region_vector.png?raw=true" alt="Region set to vector" width="200"/>

In [ ]:
%%bash
g.region -p vector=roads

However now the resolution was adjusted based on the extent of the vector map, it is no longer a nice rounded number and it doesn't align with the raster. If that's not desired, we can set set the extent based on a vector map, but **align** the resolution to a raster map:

<img src="https://github.com/ncsu-geoforall-lab/grass-gis-workshop-vanderbilt-2024/blob/main/img/region_vector_align.png?raw=true" alt="Region set to vector and aligned with raster" width="200"/>

In [ ]:
%%bash
g.region -p vector=roads align=dsm

Finally, we can save a specific region for later. This won't actually modify the current region:

In [ ]:
%%bash
g.region vector=umstead_drive_segments align=dsm save=road_region

Saved region can be used later on, for example to set the rendering extent:

In [ ]:
elevation_map = gj.Map(saved_region="road_region")
elevation_map.d_rast(map="dsm")
elevation_map.d_vect(map="umstead_drive_segments")
elevation_map.show()

## Python API

There are two Python APIs for accessing a tool's functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows. Here we will be using Python Scripting Library (`import grass.script as gs`)
as it is simple to use and sufficient for our purposes.

GRASS GIS Python Scripting Library provides functions to call GRASS modules within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file

It also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`

## Let's practice!
Compute viewshed in Python.
1. Set computational region to match raster `dsm`:

2. Compute binary viewshed (visible - 1, not visible - 0) using [r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html) using `dsm` raster from X=711260 and Y=3960860:

3. Display the viewshed overlaid over the orthophoto `ortho`. Use [d.rast](https://grass.osgeo.org/grass-stable/manuals/d.rast.html)'s `values` to display only visible cells.

4. Compute size of the visible area, using [r.univar](https://grass.osgeo.org/grass-stable/manuals/r.univar.html) computing univariate statistics. Use `gs.parse_command()` together with r.univar's `-g` flag to get a number of visible cells. Use `gs.region()` to get the cell size. Compute area in hectares (ha, 1 ha is 10,000 square meters). Compute also percentage of area that is visible.

Find answers by you editing this cell (double click to see the answers).
<!--
# 1. set computational region based on DSM
gs.run_command("g.region", raster="dsm")

# 2. Compute viewshed, flag 'b' is for binary (0 and 1) output 
gs.run_command("r.viewshed", input="dsm", output="viewshed", flags="b", coordinates=(711260, 3960860))

# 3. Display the viewshed
viewshed_map = gj.Map()
viewshed_map.d_rast(map="ortho")
# select only cells with value 1 to visualize
viewshed_map.d_rast(map="viewshed", values=1)
viewshed_map.show()

# 3. Compute viewshed size

# Compute basic univariate statistics, flag 'g' is for parsable output
univar = gs.parse_command("r.univar", map="viewshed", flags='g')
# Get current region settings to get cell size
region = gs.region()
cell_size = region["nsres"] * region["ewres"]
# Compute the percentage and size
percentage = 100 * float(univar['sum']) / float(univar['n'])
area = cell_size * float(univar['sum'])
print(f"Percentage of visible area is {percentage:.2f}%, which is {area / 10000:.2f} ha")

-->

# Part 2: Viewshed case study

In the second part, we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to **compute and analyze the area a driver would see from a road**.
This notebook can be run only after notebook Part 1 was executed.

Topics covered:
 * Python scripting
 * manipulating vector data ([v.build.polylines](https://grass.osgeo.org/grass-stable/manuals/v.build.polylines.html), [v.to.points](https://grass.osgeo.org/grass-stable/manuals/v.to.points.html))
 * vector attributes ([v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html))
 * viewshed computation ([r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html))
 * region handling ([grass.script.region_env](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region_env))
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * temporal data handling ([temporal tools](https://grass.osgeo.org/grass-stable/manuals/temporalintro.html))
 * raster mask ([r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html))
 * raster as numpy array ([grass.script.array](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#module-script.array))

In the previous notebook (Part 1) we created new project *dix_park*. This automatically created new default mapset (subproject) _PERMANENT_ where we then imported our base data. Now it's time to create a new mapset for our viewshed analysis, we will name it _viewshed_:

In [ ]:
%%bash
grass -c -e ~/grassdata/dix_park/viewshed

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata/dix_park/viewshed")

Schema of GRASS project dix_park's content:

<img src="https://github.com/ncsu-geoforall-lab/grass-gis-workshop-vanderbilt-2024/blob/main/img/data_structure2.png?raw=true" alt="GRASS project dix_park" width="400"/>

## Data preparation
We will first derive viewpoints along the road *Umstead Drive* (vector `umstead_drive_segments`) that we extracted in the first part of the workshop.

Because the road consists of several segments, we will first merge them into one.

In [ ]:
gs.run_command(
    "v.build.polylines",
    input="umstead_drive_segments",
    output="umstead_drive",
    cats="first",
)

Then create new vector of points along the line with distance 50 m:

In [ ]:
gs.run_command(
    "v.to.points", input="umstead_drive", type="line", output="viewpoints", dmax=50
)

Visualize the points with InteractiveMap with OSM tiles (see [other tile options](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html#module-grass.jupyter.interactivemap)):

In [ ]:
road_map = gj.InteractiveMap(tiles="OpenStreetMap")
road_map.add_vector("umstead_drive")
road_map.add_vector("viewpoints")
road_map.show()

Next part of analysis is raster-based, so we need to make sure we set computational region as we need. Specifically, we set it to match the DSM:

In [ ]:
gs.run_command("g.region", raster="dsm")

Now we want to compute the visibility using DSM, however some points may fall on top of a tree, so we need to filter those out.

First compute height above ground (DSM - DTM) using raster algebra:

In [ ]:
gs.mapcalc("diff = dsm - ground")

Set the color ramp of the raster to "differences", which will highlight in red areas with vegetation and buildings:

In [ ]:
gs.run_command("r.colors", map="diff", color="differences")

diff_map = gj.Map()
diff_map.d_rast(map="diff")
diff_map.d_vect(map="umstead_drive")
diff_map.d_legend(raster="diff")
diff_map.show()

Extract height above ground for the viewpoint locations to identify points that fall on top of a tree growing next to the road:

In [ ]:
gs.run_command("v.what.rast", map="viewpoints", layer=2, raster="diff", column="height")

See the newly computed attribute data. This example shows how the attribute data can be loaded into pandas:

In [ ]:
import json
import pandas as pd

pd.DataFrame(
    json.loads(
        gs.read_command(
            "v.db.select",
            map="viewpoints",
            columns="cat,height",
            layer=2,
            format="json",
        )
    )["records"]
)

Visualize the viewpoints with the height-above-ground raster. You can filter the points based on the height above ground, we won't display points with height > 2.
Additionally, we will render the result larger (`width=1000`) and we will render the map zoomed in to the area with the points
by saving a region and using it in Map (`saved_region="umstead_drive_region"`).

In [ ]:
gs.run_command(
    "g.region",
    vector="umstead_drive",
    align="dsm",
    grow=200,
    save="umstead_drive_region",
)

img = gj.Map(width=1000, saved_region="umstead_drive_region")
img.d_rast(map="diff")
img.d_vect(map="umstead_drive")
img.d_vect(
    map="viewpoints",
    layer=2,
    where="height >= 2",
    size=15,
    icon="basic/pin",
    fill_color="red",
)
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=15, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

## Viewshed computation
We will compute viewsheds from all the viewpoints we generated earlier and from those we compute a cumulative viewshed.
First, we get the list coordinates of the viewpoints that are likely lying on the ground:

In [ ]:
import csv
import io

viewpoints = gs.read_command(
    "v.out.ascii", input="viewpoints", separator="comma", layer=2, where="height < 2"
)
reader = csv.reader(io.StringIO(viewpoints))
viewpoints = list(reader)
viewpoints

We will now compute the viewshed from each viewpoint in a loop. We set max distance of 300 m. Each viewshed will be named `viewshed_{cat}`.

In [ ]:
from tqdm import tqdm

maps = []
for x, y, cat in tqdm(viewpoints):
    name = f"viewshed_{cat}"
    gs.run_command(
        "r.viewshed",
        input="dsm",
        output=name,
        coordinates=(x, y),
        max_distance=300,
        flags="b",
    )
    maps.append(name)

In [ ]:
img = gj.Map(width=1000, saved_region="umstead_drive_region")
img.d_rast(map="diff")
for name in tqdm(maps):
    img.d_rast(map=name, values=1)
img.d_vect(map="umstead_drive")
img.d_vect(
    map="viewpoints",
    layer=2,
    where="height >= 2",
    size=15,
    icon="basic/pin",
    fill_color="red",
)
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=15, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

## Temporal dataset of viewsheds

In this part we will create, analyze and visualize a temporal dataset of viewsheds using [temporal tools](https://grass.osgeo.org/grass-stable/manuals/temporal.html). 

First, let's check we have the viewshed rasters ready:

In [ ]:
gs.list_strings(type="raster", pattern="viewshed_*")

We will create an empty space-time raster dataset called _viewsheds_ with relative temporal type:

In [ ]:
gs.run_command(
    "t.create",
    output="viewsheds",
    type="strds",
    temporaltype="relative",
    title="Viewshed series",
    description="Series of viewsheds along a road",
)

Now we register the viewshed rasters with start time 1 and 1-minute increment to simulate a change of view of a car driving slowly along the road:

In [ ]:
gs.run_command(
    "t.register",
    input="viewsheds",
    maps=",".join(maps),
    start=1,
    unit="minutes",
    increment=1,
)

Let's print basic dataset info. We will use this info later on to set computational region covering the entire dataset.

In [ ]:
info = gs.parse_command("t.info", input="viewsheds", flags="g")
pd.DataFrame(info.values(), index=info.keys())

To list the individual rasters, we will use t.rast.list.

In [ ]:
pd.read_csv(
    io.StringIO(
        gs.read_command(
            "t.rast.list",
            input="viewsheds",
            separator="comma",
            columns="name,start_time",
        )
    )
)

We can quickly get basic statistics such as the size of the viewsheds (see _sum_ column for the number of visible cells):

In [ ]:
df = pd.read_csv(
    io.StringIO(gs.read_command("t.rast.univar", input="viewsheds", separator="comma"))
)
df

Let's find and visualize largest and smallest viewshed:

In [ ]:
largest = df.iloc[df[["sum"]].idxmax()["sum"]].id
smallest = df.iloc[df[["sum"]].idxmin()["sum"]].id

viewshed_map = gj.Map(saved_region="umstead_drive_region")
viewshed_map.d_rast(map="ortho")
viewshed_map.d_rast(map=largest, values=1)
viewshed_map.d_rast(map=smallest, values=1)
viewshed_map.d_vect(map="umstead_drive", color="white")
viewshed_map.show()

Let's compute a temporal dataset where values of each viewshed will represent the registered start time.

We use temporal raster algebra. Here we compute a new temporal dataset _viewsheds_start_ so that for example viewshed with start time 5 has value 5 for visible area and no data for invisible area.

In [ ]:
gs.run_command(
    "t.rast.mapcalc",
    inputs="viewsheds",
    output="viewsheds_start",
    basename="viewshed_start",
    expression="if (viewsheds == 0, null(), start_time())",
)

Set color of the newly computed time series:

In [ ]:
gs.run_command("t.rast.colors", input="viewsheds_start", color="plasma")

With TimeSeriesMap, we can interactively visualize the time series:

In [ ]:
timemap = gj.TimeSeriesMap(width=800)
timemap.d_rast(map="ortho")
timemap.d_vect(map="umstead_drive")
timemap.add_raster_series("viewsheds_start")
timemap.show()

We can export an animated GIF:

In [ ]:
from IPython.display import Image

Image(timemap.save("animation.gif", duration=300))

## Cumulative viewshed
We can compute the cumulative viewshed, which aggregates viewsheds from multiple viewpoints. In this way you can e.g., identify the most frequently visible areas from the road.


Since our viewshed rasters are binary (0 invisible, 1 visible), we will use r.series method *sum*. Then we replace zeros with no data using r.null and set a new color ramp:

In [ ]:
# cumulative viewshed
gs.run_command("r.series", input=maps, output="cumulative_viewshed", method="sum")
gs.run_command("r.null", map="cumulative_viewshed", setnull=0)
gs.run_command("r.colors", map="cumulative_viewshed", color="plasma")

Let's visualize the results:

In [ ]:
cumulative_map = gj.InteractiveMap()
cumulative_map.add_raster("cumulative_viewshed", opacity=0.8)
cumulative_map.add_vector("umstead_drive")
cumulative_map.add_layer_control(position="bottomright")
cumulative_map.show()

And create a 3D rendering with draped cumulative viewshed over the DSM:

In [ ]:
map3d = gj.Map3D()
map3d.render(
    elevation_map="dsm",
    resolution_fine=1,
    color_map="cumulative_viewshed",
    vline="umstead_drive",
    vline_width=3,
    vline_color="white",
    light_brightness=50,
    position=[0.4, 0.8],
    height=3000,
    perspective=10,
)
map3d.overlay.d_legend(
    raster="cumulative_viewshed", at=(0, 30, 1, 7), use=[1, 2, 3, 4, 5, 6], flags="fb"
)
map3d.show()

## Mask
Now let's compute Excess Green Index (ExGI, [Woebbecke et al.](https://elibrary.asabe.org/abstract.asp?aid=27838)) from orthophoto and analyze its distribution within the visible area.

Compute ExGI using raster algebra:

In [ ]:
gs.run_command("r.rgb", input="ortho", red="red", green="green", blue="blue")
gs.mapcalc("exgi = (2.0 * green - red - blue)")

In [ ]:
ndvi_map = gj.Map()
ndvi_map.d_rast(map="exgi")
ndvi_map.d_legend(raster="exgi", flags="t")
ndvi_map.show()

We will mask the data by the visible area:

In [ ]:
gs.run_command("r.mask", raster="cumulative_viewshed")
data = gs.parse_command("r.univar", map="exgi", flags="g")
print(
    f"Average ExGI of visible cells: {float(data['mean']):.2f} ± {float(data['stddev']):.2f}"
)

In [ ]:
ndvi_map = gj.Map()
ndvi_map.d_rast(map="exgi")
ndvi_map.d_legend(raster="exgi", flags="t")
ndvi_map.show()

Let's see the histogram of visible ExGI using d.histogram:

In [ ]:
histo = gj.Map(width=800, height=400)
histo.d_histogram(map="exgi", bgcolor="#EAEAF2")
histo.show()

## Read as numpy array
It is also easy to use the results as a numpy array and then use other Python libraries to analyze the data:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

In [ ]:
exgi = garray.array(mapname="exgi", null="nan")
exgi

In [ ]:
sns.set_style("darkgrid")
sns.histplot(exgi.ravel(), kde=True)
plt.show()

Finally, remove the mask:

In [ ]:
gs.run_command("r.mask", flags="r")